<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/11-integrate-cimg-library/Refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3y5rb5k5".
Wed Apr  3 20:12:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                 

In [ ]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev
!apt-get install -y libx11-dev

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/GPU Computing/human detection dataset'

0  1


In [ ]:
!git clone --recursive https://github.com/YoungYoung619/pedestrian-detection-in-hazy-weather.git

In [ ]:
from PIL import Image
import os

def get_image_dimensions(folder_path):
    image_dimensions = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                with Image.open(file_path) as img:
                    width, height = img.size
                    image_dimensions.append((filename, width, height))
            except:
                print(f"Error processing {filename}")

    return image_dimensions

folder_path = '/content/drive/My Drive/GPU Computing/human detection dataset/0'
dimensions = get_image_dimensions(folder_path)
for image in dimensions:
    print(f"Image: {image[0]}, Dimensions: {image[1]}x{image[2]}")


In [ ]:
from PIL import Image
import os

def convert_png_to_jpeg(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.jpg')
        if os.path.isfile(input_file_path):
            try:
                # Open the image
                with Image.open(input_file_path) as img:
                    # Convert PNG to JPEG
                    if img.format == 'PNG':
                        img.convert('RGB').save(output_file_path, 'JPEG')
                        print(f"Converted {filename} to JPEG")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Specify the input and output folders
input_folder = '/content/drive/My Drive/GPU Computing/human detection dataset/0'
output_folder = '/content/human detection dataset/0_jpeg'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert PNG images to JPEG format
convert_png_to_jpeg(input_folder, output_folder)


In [ ]:
!ls '/content/human detection dataset/0_jpeg'

In [11]:
!wget -q https://github.com/dtschump/CImg/archive/refs/tags/v.2.9.6.tar.gz
!tar -xf v.2.9.6.tar.gz


mv: cannot stat 'CImg-2.9.6': No such file or directory


In [14]:
!mv "CImg-v.2.9.6" "CImg"

In [39]:
%%writefile read_image.cpp
#include <iostream>
#include "CImg/CImg.h"

using namespace cimg_library;

int main() {
    // Create an instance of the CImg class to hold the image
    CImg<unsigned char> image;

    // Load the image file
    const char* filename = "image.jpeg";

    try {
        image.load(filename);
    } catch (CImgIOException& e) {
        std::cerr << "Error loading image: " << e.what() << std::endl;
        return 1;
    }

    // Display some information about the image
    std::cout << "Image dimensions: " << image.width() << "x" << image.height() << std::endl;
    std::cout << "Number of channels: " << image.spectrum() << std::endl;

    // Optionally, display the image
    CImgDisplay display(image, "Loaded Image");
    while (!display.is_closed()) {
        display.wait();
    }

    return 0;
}


Overwriting read_image.cpp


In [40]:
!g++ read_image.cpp -o read_image -lpthread -lX11
!./read_image



[CImg] *** CImgIOException *** [instance(0,0,0,0,(nil),non-shared)] CImg<unsigned char>::load(): Failed to recognize format of file 'image.jpeg'.
Error loading image: [instance(0,0,0,0,(nil),non-shared)] CImg<unsigned char>::load(): Failed to recognize format of file 'image.jpeg'.


In [62]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <fstream>
#include <sstream>
#include <iostream>
#include <vector>
#include <filesystem>

#define CUDA_CHECK_ERROR(value) {											\
		cudaError_t _m_cudaStat = value;										\
		if (_m_cudaStat != cudaSuccess) {										\
			fprintf(stderr, "Error %s at line %d in file %s\n",					\
					cudaGetErrorString(_m_cudaStat), __LINE__, __FILE__);		\
					exit(1);															\
		}
}

__global__ void CUDA::computeGradient(const Image3* _img, Vec2* _imgGradient) {
	//TODO:  handle leftovers. implement shared mem
	if (threadIdx.y < blockDim.y && threadIdx.x < blockDim.x) {

		uint2 blkShr;	// Each block's share //-2 to remove border padding
		blkShr.x = (_img->width() - 2 * BORDERTHICKNESS) / gridDim.x;
		blkShr.y = (_img->height() - 2 * BORDERTHICKNESS) / gridDim.y;
		uint2 tdShr; //Each thread's share;
		tdShr.x = blkShr.x / blockDim.x;
		tdShr.y = blkShr.y / blockDim.y;
		uint2 idx;
		idx.x = blockIdx.x * blkShr.x + threadIdx.x * tdShr.x;
		idx.y = blockIdx.y * blkShr.y + threadIdx.y * tdShr.y;

		//		 Gradient computation -1 0 1 in x and y
		float2* imgGradientPtr;
		uchar3 *prevImgPtr = _img->getConstPtr(1, idx.y), *imgPtr =
				_img->getConstPtr(1, idx.y + 1), *nextImgPtr;
		float gxR, gxG, gxB, gyR, gyG, gyB, mag, mR, mG, mB;
		int x = idx.x, y = idx.y;
	}
  return;
}

int main() {

    std::string image_path = "/content/drive/My Drive/GPU Computing/human detection dataset/0/117.png"; // Change this to your folder path
    cv::Mat imageBeforeResize = cv::imread(image_path, cv::IMREAD_GRAYSCALE);
    cv::Mat image;
    cv::resize(imageBeforeResize, image, cv::Size(256, 256)); // Resize to standard size
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
    }
    unsigned char* d_image;
    size_t imageSize = image.total() * image.elemSize();
    CUDA_CHECK_ERROR(cudaMalloc(&d_image, imageSize));
    CUDA_CHECK_ERROR(cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice));

    size_t sizeInBytes = image.total() * sizeof(float);
    float* d_magnitude;
    CUDA_CHECK_ERROR(cudaMalloc((void **)&d_magnitude, sizeInBytes));

    CUDA_CHECK_ERROR(cudaMemset(d_magnitude, 0, sizeInBytes));

    //dim3 gridSize = dim3(20, 20, 1);
    //dim3 blockSize = dim3(32, 24, 1);

    return 0;
}


Overwriting gradient_computation.cu


In [63]:
!nvcc gradient_computation.cu -o gradient_computation -diag-suppress 611 `pkg-config --cflags --libs opencv4` -run


gradient_computation.cu(15): error: expected a declaration
  }
  ^

gradient_computation.cu(38): warning #12-D: parsing restarts here after previous syntax error
    return;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

gradient_computation.cu(39): error: expected a declaration
  }
  ^

gradient_computation.cu(55): warning #12-D: parsing restarts here after previous syntax error
      size_t sizeInBytes = image.total() * sizeof(float);
                                                        ^

gradient_computation.cu(57): error: expected a declaration
      { cudaError_t _m_cudaStat = cudaMalloc((void **)&d_magnitude, sizeInBytes); if (_m_cudaStat != cudaSuccess) { fprintf(
      ^

3 errors detected in the compilation of "gradient_computation.cu".


In [ ]:
!./gradient_computation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Assuming df is your DataFrame with features and labels
# Splitting data into features (X) and labels (y)

df = pd.read_csv("descriptor.csv")
new_value = 0

df.iloc[-1, df.columns.get_loc('label')] = new_value

X = df.drop(columns=['label'])  # Replace 'target_column' with the name of your target column
y = df['label']


In [ ]:
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,label
0,0.644542,0.270649,0.209346,0.219895,0.243296,0.205410,0.306049,0.317417,0.351183,0.425772,...,0.227495,0.164700,0.247566,0.386196,0.698530,0.371040,0.199949,0.164022,0.134855,1
1,0.489285,0.313330,0.338145,0.378378,0.395283,0.272598,0.267245,0.266543,0.178574,0.235455,...,0.344043,0.331950,0.361637,0.300967,0.361063,0.318854,0.367762,0.309161,0.295327,1
2,0.603244,0.262909,0.288703,0.275478,0.254339,0.257944,0.336061,0.273818,0.297660,0.380624,...,0.430357,0.329545,0.352977,0.268352,0.402616,0.313292,0.241961,0.260699,0.350467,1
3,0.688987,0.268127,0.223381,0.217443,0.241237,0.232688,0.271446,0.276305,0.306363,0.418343,...,0.263961,0.188683,0.210332,0.207616,0.385607,0.262730,0.543682,0.392910,0.373754,1
4,0.575349,0.291004,0.272408,0.250846,0.255007,0.317185,0.290041,0.332377,0.294834,0.293702,...,0.343681,0.257530,0.296350,0.317993,0.333191,0.291995,0.419408,0.368308,0.344657,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.406344,0.115792,0.111428,0.116107,0.839464,0.121537,0.117669,0.129851,0.213078,0.271225,...,0.230496,0.180408,0.135713,0.344164,0.678173,0.508680,0.101348,0.128973,0.178546,0
518,0.410489,0.093417,0.097809,0.255268,0.694336,0.433774,0.179545,0.144973,0.156615,0.357678,...,0.336548,0.128620,0.203286,0.252332,0.420822,0.561934,0.138108,0.127756,0.486764,0
519,0.888679,0.030741,0.044936,0.031906,0.391852,0.044424,0.030935,0.032400,0.220770,0.670775,...,0.577010,0.249210,0.216837,0.263973,0.359025,0.437901,0.263333,0.227103,0.216050,0
520,0.790527,0.038492,0.051095,0.054766,0.370023,0.064761,0.124946,0.107843,0.446791,0.508999,...,0.456539,0.209702,0.103131,0.375936,0.406504,0.181839,0.176525,0.218455,0.564303,0


In [ ]:
y

0    1
1    1
2    1
3    0
Name: label, dtype: int64

In [ ]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Linear Support Vector Machine
svm = LinearSVC()
svm.fit(X_train, y_train)

# Making predictions
y_pred = svm.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6857142857142857
